<a href="https://colab.research.google.com/github/bilgeyucel/presentations/blob/main/aws-community-day-2024/implement_fallback_to_rag_with_Amazon_Bedrock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Building Fallbacks to Websearch with Conditional Routing

- **Level**: Beginner
- **Time to complete**: 10 minutes
- **Components Used**: [`ConditionalRouter`](https://docs.haystack.deepset.ai/v2.0/docs/conditionalrouter), [`SerperDevWebSearch`](https://docs.haystack.deepset.ai/v2.0/docs/serperdevwebsearch), [`PromptBuilder`](https://docs.haystack.deepset.ai/v2.0/docs/promptbuilder), [`OpenAIGenerator`](https://docs.haystack.deepset.ai/v2.0/docs/openaigenerator)
- **Prerequisites**: You must have an [Open API Key](https://platform.openai.com/api-keys) and a [Serper API Key](https://serper.dev/api-key) for this tutorial
- **Goal**: After completing this tutorial, you'll have learned how to create a pipeline with conditional routing

> This tutorial uses Haystack 2.0 Beta. To learn more, read the [ Haystack 2.0 Beta announcement](https://haystack.deepset.ai/blog/introducing-haystack-2-beta-and-advent) or visit the [Haystack 2.0 Documentation](https://docs.haystack.deepset.ai/v2.0/docs).


## Overview

When developing applications using **retrieval augmented generation ([RAG](https://www.deepset.ai/blog/llms-retrieval-augmentation))**, the retrieval step plays a critical role. It serves as the primary information source for **large language models (LLMs)** to generate responses. However, if your database lacks the necessary information, the retrieval step's effectiveness is limited. In such scenarios, it may be practical to use the web as a fallback data source for your RAG application. By implementing a conditional routing mechanism in your system, you gain complete control over the data flow, enabling you to design a system that can leverage the web as its data source under some conditions.

In this tutorial, you will learn how to create a pipeline with conditional routing that directs the query to a **web-based RAG** route if the answer is not found in the initially given documents.

## Development Environment

Install Haystack and `amazon-bedrock-haystack` with `pip`:

In [ ]:
%%bash

pip install haystack-ai amazon-bedrock-haystack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.5/319.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00


### Enter API Keys

Enter API keys required for this tutorial.

In [ ]:
from getpass import getpass
import os
from google.colab import userdata

os.environ["SERPERDEV_API_KEY"] = userdata.get('SERPERDEV_API_KEY') or getpass("Enter Serper Api key: ")

In [ ]:
import os
from getpass import getpass

os.environ["AWS_ACCESS_KEY_ID"] = getpass("aws_access_key_id: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass("aws_secret_access_key: ")
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

aws_access_key_id: ··········
aws_secret_access_key: ··········


## Creating the Documents

there are 5 documents scraped from the web about Taylor Swift ✨

In [ ]:
from haystack.dataclasses import Document

documents = [
Document(
    content = """
For two female journalists in their 30s - who also happen to be massive Swifties - there's a lot about Taylor Swift's new album that rings true.
From exes who strung us along, to comfort-eating after a breakup. We've all been there, and pop's biggest superstar has too.
Swift is no stranger to writing about personal subject matters. And she's also by no means the first musician to sing about heartbreak, pain and sorrow.
But in The Tortured Poets Department, Swift pinpoints the unique 21st Century anxieties that so many of us millennials have experienced when dating.
Perhaps more than any other song on her new album, So Long, London is the real sucker punch.
"I'm pissed off you let me give you all that youth for free," she laments, in a track widely thought to be about her ex-partner, Joe Alwyn.
This feels like a pivotal moment in the album. A moment so raw, that you're stopped in your tracks.
It doesn't matter that Swift is a world-famous musician, with A-list friends and a massive billion-dollar fortune. Beneath all of that, she's a 34-year-old woman, who understands all too well the anxieties about running out of time to find "The One", settle down and start a family.
Rebecca Reid Rebecca ReidRebecca Reid
Rebecca Reid says Taylor Swift has taken her "through good times and bad times"
Rebecca Reid, a Swiftie in her early 30s, told BBC News it felt like The Tortured Poets Department could have been written for her in mind.
"With So Long, London, but honestly, almost every single song, there's so much on there about the idea that you gave somebody your youth and that you can't get that back," she said.
"And that's definitely a feeling that I really resonate with."
In another song, Take Down Bad, Swift sings: "Now I'm down bad, crying at the gym."
Again, these are lyrics that strike a chord for many. Who hasn't experienced the depression of a breakup, which leaves you in tears as you try to go about your everyday routine?
Other lyrics
Saira Thwaites Saira ThwaitesSaira Thwaites
Saira Thwaites has grown up with Taylor Swift
For Saira Thwaites, who's almost 30 and a committed Swiftie, the more she listens to the tracks, the more she can relate to them.
"Her stories are so specific, and they really sum up the numbness and emptiness of a breakup," she says.
Even when Swift's on stage, surrounded by adoring fans during her Eras tour - a tour so successful that it's made her a billionaire - her grief remains in place.
"Breaking down, I hit the floor / All the pieces of me shattered as the crowd was chanting, 'More'," she sings on the deceitfully upbeat I Can Do It With A Broken Heart.
'Swift still experiences dating despair'
"[The song] is about telling everybody you're fine and being creative and pushing through when you're not really giving yourself the space to heal or to grieve, that you need this," Reid says.
"Again, that's something that I can really resonate with because I spent the early period of my break-up single parenting, and going on TV and radio and writing books and telling everybody how great I was and how happy I was when I was, in fact, processing one of the worst traumas of my life."
The List: 13 Times Taylor Swift Made History
We're bringing up Taylor Swift's history: from dizzying Grammy records to reaching billionaire status and everything in between.
Watch now on BBC iPlayer (UK only)
BBC iPlayer logo
Helen Brown, a music critic at The Independent, says "a whole generation of women" have found Swift's songs to be the soundtrack to their lives.
"Singing of the elusive lure of rings and cradles, Swift articulates the challenges facing a generation who are marrying and having children on average five years later than in the 1990s," she tells BBC News.
"It's equally reassuring and alarming to think that even without the financial challenges facing most people her age, Swift still experiences the dating despair of her peers.
"Like them she sounds overwhelmed by the options and describes being ghosted as she asks herself if she expects too much, or too little of herself and her partners."
Getty Images Taylor Swift and Joe AlwynGetty Images
Taylor Swift and Joe Alwyn at the 2020 Golden Globe Awards
In telling the story of modern dating, Swift has never held back in writing about her exes.
Many are reading her latest album as a dig in particular at Alwyn and the 1975's Matty Healy while also touching on her current sweetheart, NFL superstar Travis Kelce.
Her intentions are laid bare from the sleeve notes for the album, in which she says: "A smirk creeps onto this poet's face. Because it's the worst men that I write best."
Swift and Alwyn, an actor, split up in April 2023. When she later announced the arrival of a new album, fans immediately began to speculate it would deal with the fall-out.
Her choice of album title echoed a WhatsApp group chat that Alwyn and Normal People star Paul Mescal had, called The Tortured Man Club, adding to the speculation.
In So Long, London, she hints at wedding plans, singing: "You swore that you loved me, but where were the clues, I died on the altar waiting for the proof."
She also revealed she was upset at having to leave London, where she'd lived with Alwyn - adding that she'd "loved" the city.
Another track, But Daddy I Love Him, is also thought to address the discourse that surrounded Swift's reported-but-never-confirmed romance with The 1975's lead singer Healy, last year.
Some fans felt let down by the relationship, saying that Healy - who has faced accusations of misogyny and racism in his career (all of which he denies) - was an inappropriate choice of partner.
In her song, Swift hits back, declaring: "I'd rather burn my whole life down than listen to one more second of all this bitching and moaning / I'll tell you something about my good name, it's mine alone to disgrace."
Getty Images Travis Kelce and Taylor Swift celebrate the Chiefs' Super Bowl winGetty Images
Taylor Swift is now in a relationship with US football star Travis Kelce
But is humiliating your exes in public the right thing to do? Brown says it is "a complicated issue".
"Swift doesn't name anybody in these songs and her real history has always been braided with fiction. She's a storyteller, coming from a country music tradition which has a long history of female stars calling out bad behaviour from men," she says.
"I'd add that while Swift may take swipes at her exes, she always holds herself to account. As both the exes she appears to be addressing on this record are also songwriters, they've got a right to reply in their own work and I suspect they both think all's fair in love and lyrics."
The BBC has reached out to both Alywn - who did not respond - and Healy, who wasn't available for comment.
So where does all of that leave Swift's current very public relationship with Kelce?
'Love makes Taylor behave like a kid'
Nona Uppal, another dedicated Swiftie, tells the BBC that while a lot of The Tortured Poets Department is about despair and heartbreak, it also nods to the happiness Swift feels in a new relationship, which is also something that many people can identify with.
Nona Uppal Nona UppalNona Uppal
Nona Uppal is an author and self-declared Swiftie
She points to the song So High School, which she interprets as being about Kelce as it is "all about butterflies and doing the things that kids would do in watching movies with your friends".
"I think it just captures a whole spectrum of human emotion where love makes Taylor behave like a kid. And that to her is what love is all about," she says.
"And I just love the level of vulnerability that captures, because I think that's something I relate to pretty heavily."
""",
meta={"url": "https://www.bbc.com/news/entertainment-arts-68856232", "title": "Tortured Poets Department: How Taylor Swift album lyrics captured modern dating despair"}
),
Document(
    content="""
Taylor Swift gave fans a lot more than they bargained for at the 2024 Grammys, dropping the bombshell news mid-acceptance speech that her new album, The Tortured Poets Department, is arriving this year.
The welcome news came as the pop star was accepting best pop vocal for her 2022 record Midnights — aka, the last album she announced without warning during an awards show acceptance speech. “This is my 13th Grammy – which is my lucky number, I don’t know if I’ve ever told you that,” she teased the crowd from the stage at Crypto.com Arena Sunday (Feb. 4). “I want to say thank you to the fans by telling you a secret that I’ve been keeping from you for the last two years …”
Swift went on to reveal the pending arrival of The Tortured Poets Department before posting the album’s sultry cover almost immediately after she left the stage. “All’s fair in love and poetry…,” she captioned the artwork. “New album THE TORTURED POETS DEPARTMENT.”
Later that night, the 34-year-old singer-songwriter took home a historic fourth album of the year for Midnights, but Swifties were probably more excited for the new LP announcement, especially because most were expecting that the musician would announce her long-awaited Reputation (Taylor’s Version) at the ceremony instead — so much so that “Rep TV” was trending ahead of the Grammys. Swift has been steadily releasing re-recorded versions of her first six albums since 2021. Following Fearless, Red, Speak Now and 1989, she only has her self-titled debut and Rep left to go — but clearly, fans will just have to wait a little longer as the Tortured Poets era takes hold.
From the release date to the song titles, see everything Billboard knows about Swift’s 11th album The Tortured Poets Department below.
The Title
Obviously, the album is called The Tortured Poets Department — but fans suspect there are layers to that title. Namely, Swifties were quick to note how similar it sounds to a certain group chat shared by Swift’s ex-boyfriend Joe Alwyn and Paul Mescal, as revealed by the two actors in a December 2022 Variety interview: “The Tortured Man Club.”
The Cover
In the black-and-white album cover shared by Swift moments after announcing the album, she poses in a black top and shorts with her arms wrapped around her body, lying back on white bedding. Similar to the original 1989 cover, the top half of her face is obstructed.
Stylist Joseph Cassell revealed on Instagram that the photo features Swift wearing a black cami tank top and matching high-rise briefs from Mary Kate and Ashley’s fashion brand The Row.
When It's Coming
As revealed by Swift in her Grammys speech, The Tortured Poets Department arrives Friday, April 19.
The Tracklist
Swift shared the album’s full tracklist one day after announcing the project, revealing that Tortured Poets has a total of 16 songs plus one bonus track:
“Fortnight”
“The Tortured Poets Department”
“My Boy Only Breaks His Favorite Toys”
“Down Bad”
“So Long, London”
“But Daddy I Love Him”
“Fresh Out the Slammer”
“Florida!!!!”
“Guilty as Sin?”
“Who’s Afraid of Little Old Me?”
“I Can Fix Him (No Really I Can)”
“loml”
“I Can Do It With a Broken Heart”
“The Smallest Man Who Ever Lived”
“The Alchemy”
“Clara Bow”
Bonus Track: “The Manuscript”
The Collaborators
The Tortured Poets Department tracklist featured two major collaborations: Post Malone, whose name is billed on track one, “Fortnight,” as well as Florence + The Machine on track eight, “Florida!!!”
Both Posty and Florence Welch have sung Swift’s praises in the past, with the rapper calling the pop star “a great f—in’ songwriter” in an October 2023 interview with Howard Stern, and Welch and Swift trading compliments in a 2015 Billboard interview.
“What sets Florence apart? Everything,” Swift said at the time. “Every time I’ve been around her, she is the most magnetic person in the room — surrounded by people who are fascinated by the idea of being near her.”
A Possible Lyric
While sharing the album’s cover art to Instagram, Swift also included a photo of what appeared to be a handwritten lyric from one of the project’s tracks. “And so I enter into evidence/ My tarnished coat of arms,” reads the message, penned in black pink on white stationary labeled “TTPD.”
“My muses, acquired like bruises/ My talismans and charms/ The tick, tick, tick of love bombs,” it continues. “My veins of pitch black ink.”
“All’s fair in love and poetry …,” Swift signed off the note. “Sincerely, The Chairman of The Tortured Poets Department.”
Travis Kelce Approves
Swift’s superstar boyfriend, Kansas City Chiefs tight end Travis Kelce, revealed in a pre-Super Bowl press interview that he got to be one of the singer’s beta listeners for The Tortured Poets Department — and his review was a rave.
“I have heard some of it, yes, and it’s unbelievable,” he told Extra on the field at Allegiant Stadium in Las Vegas. “I can’t wait for her to shake up the world when it finally drops.”
How Long It's Been in the Works
During her first Eras Tour show in Japan on Feb. 7 — just days after first announcing Tortured Poets — Swift revealed that the project had been in the making for a little while. “I’ve been working on it for about two years,” she told her crowd at Tokyo Dome. “I kept working on it throughout the U.S. tour, and when it was perfect — in my opinion when it’s good enough for you — I finished it and I am so, so excited that soon you’ll get to hear it.”
She also noted, “I’ve been working on Tortured Poets since right after I turned in Midnights,” which would’ve been a few months before the latter album dropped in October 2022.
'The Bolter' Edition
Swift announced a second version of Tortured Poets dubbed “The Bolter” during her Feb. 16, 2024, Eras Tour stop in Melbourne, complete with an alternate cover that reads “You don’t get to tell me about sad” on the back.
In place of “The Manuscript,” she confirmed that this edition of the album will feature a bonus track also titled “The Bolter.”
It Was Therapeutic to Write
While discussing the album during her Feb. 16, 2024, show in Melbourne — the same one she announced “The Bolter” special edition — Swift revealed that Tortured Poets is her most cathartic project yet.
“It kind of reminded me of why songwriting is something that actually gets me through my life,” she shared, sitting at her floral-painted surprise song piano. “I’ve never had an album where I needed songwriting more than I needed it on Tortured Poets.”
'The Albatross' Edition
About a week after announcing the “Bolter” edition of Tortured Poets, Swift unveiled another Tortured Poets installment called “The Albatross” during her Feb. 23, 2024, show in Sydney, Australia. Again, the third version of the album features its own cover and limited bonus track.
Merriam-Webster defines the term “albatross” as “large web-footed seabirds that have long slender wings, are excellent gliders, and include the largest seabirds” as well as “something that causes persistent deep concern or anxiety.”
“Am I allowed to cry?” reads the back cover.
'The Black Dog' Edition
At her March 3 Eras Tour performance in Singapore, Swift announced the fourth — and final — version of Tortured Poets. Titled “The Black Dog,” the issue’s cover features a black-and-white portrait of the singer dramatically wrapping her arms around her head.
Complete with its own bonus track, the “Black Dog” edition’s tagline is as follows: “Old habits die screaming…”
    """,
    meta={"url":"https://www.billboard.com/lists/taylor-swift-the-tortured-poets-department-everything-we-know/the-black-dog-edition/", "title":"Everything We Know About Taylor Swift’s New Album ‘The Tortured Poets Department’ So Far"}
),
Document(
    content="""
Taylor Swift is entering a new music era!
During the 2024 Grammy Awards, the “End Game” singer made the surprise announcement that she’s releasing her eleventh studio album, during her acceptance speech for best pop vocal album for Midnights. (The win also marked her 13th Grammy award, famously her lucky number.
During the speech, Swift thanked her fans for their support of her 10th album, Midnights, and said she wanted to repay them by telling them a secret she’s been keeping for two years: the details of her brand-new album.
Immediately after she left the stage, her social media accounts shared more information on the upcoming drop, including the cover art, the title The Tortured Poets Department, and the tagline, "All’s fair in love and poetry.”
Since then, Swift has shared more details about the album ahead of its April release, including sharing the official tracklist and announcing the A-list stars joining her for two songs.
Most recently, she teased that heartbreak would be a major theme of the album as she released five exclusive playlists with Apple Music that explore the five stages of a breakup: denial, anger, bargaining, depression and acceptance.
Ahead, here’s everything to know about Swift’s new album.
All the Ways Taylor Swift Teased The Tortured Poets Department Ahead of Its Announcement
What is Taylor Swift’s new album titled?
During her Grammys speech, Swift revealed that her new album will be titled The Tortured Poets Department. Directly after, she shared the cover art for the album, which showcases a black and white image of Swift wearing a blank tank and shorts on a bed.
How many songs will Taylor Swift’s The Tortured Poets Department album have?
According to Swift’s website, the new album will feature 16 songs. She is also releasing four variants of the album which will each include one bonus track, including "The Manuscript," "The Bolter," “The Albatross” and "The Black Dog."
What is the tracklist for Taylor Swift’s The Tortured Poets Department album?
The day after the album's announcement, Swift unveiled the official tracklist for the album, which has caused many Swifties to theorize about the titles.
Side A includes “Fortnight," “The Tortured Poets Department,” “My Boy Only Breaks His Favorite Toys,” and “Down Bad.” The songs on Side B are “So Long, London,” “But Daddy I Love Him,” “Fresh Out the Slammer,” and “Florida!!!.”
Side C includes “Guilty as Sin?,” “Who's Afraid of Little Old Me?,” “I Can Fix Him (No Really I Can),” and “loml.”
The album's final side features “I Can Do It With a Broken Heart,” “The Smallest Man Who Ever Lived,” “The Alchemy,” “Clara Bow."
As previously noted, there will also be four bonus tracks, "The Manuscript," "The Bolter," “The Albatross” and "The Black Dog."
Clara Bow's Family Reacts to Taylor Swift’s New Song, Says Actress Would Have Been 'Protective' of Singer (Exclusive)
What will Taylor Swift’s The Tortured Poets Department album be about?
Fans have theorized that the upcoming album will detail Swift's split from actor Joe Alwyn, whom she dated for six years before splitting in April 2023. Not only do the song titles include references to the English actor and their former relationship, but Swift recently teased that the album is about heartbreak as she shared five exclusive playlists with Apple Music.
According to a press release, with each playlist, Swift has chosen songs from her own catalog that fit each stage of a breakup: denial, anger, bargaining, depression and acceptance.
Ahead of the total solar eclipse on April 8, Swift teased the first lyric from the album as she posted a video of a typewriter typing, "Crowd goes wild at her fingertips/Half moonshine, Full eclipse."
Who will be featured on Taylor Swift’s The Tortured Poets Department album?
When Swift announced the tracklist, she also revealed which two artists will be featured on the album. including Post Malone on "Fortnight" and Florence + The Machine on “Florida!!!"
It's unclear who the musical collaborators will be on Swift's new album, however, it's likely she will work with some of her close friends such as Jack Antonoff.
How to purchase Taylor Swift’s The Tortured Poets Department album?
Shortly after announcing the album on her socials, Swift made the album available for preorder on her website. Fans can purchase the album on CD, cassette, vinyl and digital.
There will also be a Target exclusive version of the album, including a Phantom Clear exclusive vinyl that will feature a "collectible 24-page book-bound jacket with three replicas of Taylor Swift’s handwritten lyrics," per a press release.
Additionally, Target is releasing three exclusive CDs, which will feature a "2-sided poster with a photo and replica of Taylor’s handwritten lyrics, a 20-page booklet, unique front and back cover, a bonus track and more," per a press release.
Will there be a deluxe version of Taylor Swift’s The Tortured Poets Department album?
In addition to the standard album, Swift is releasing four deluxe versions of the album, each of which will feature a different bonus track.
When is Taylor Swift releasing The Tortured Poets Department?
Swift revealed that the new album is slated to drop on April 19, 2024. The release comes right in the middle of Swift's successful Eras Tour, where she takes fans through her extensive music catalog. However, it's unclear how Swift's new album will affect that setlist.
What time will The Tortured Poets Department be released?
Like Swift's previous albums, the album will drop at midnight ET on Friday, April 19, so set your clocks!
""",
    meta={"url": "https://people.com/taylor-swift-the-tortured-poets-department-album-everything-to-know-8558643", "title": "Everything to Know About Taylor Swift's New Album, The Tortured Poets Department"}
),
Document(
    content="""
Taylor Swift's upcoming album, The Tortured Poets Department, is here, and it turns out it's a double album, with 31 songs. Swift surprised fans at 2 am Eastern by revealing an extra 15 songs to go along with the 16 already expected.
"Tt's a 2am surprise: The Tortured Poets Department is a secret DOUBLE album," Swift wrote on Instagram. "I'd written so much tortured poetry in the past 2 years and wanted to share it all with you, so here's the second installment of TTPD: The Anthology. 15 extra songs. And now the story isn't mine anymore… it's all yours."
The day before the big event, the singer revealed the first single off that now-double album. According to Swift's Instagram post, the first single is called Fortnight, and it features rapper Post Malone.
"I've been such a huge fan of Post because of the writer he is, his musical experimentation and those melodies he creates that just stick in your head forever," Swift wrote on Instagram. "I got to witness that magic come to life firsthand when we worked together on Fortnight. Honestly can't wait for you guys to hear this song at midnight TONIGHT and see the video at 8 pm ET TOMORROW."
Swift announced the April 19 release of her next album while accepting the award for best pop vocal album at the Feb. 4 Grammy Awards. She's subsequently offered multiple special editions of the new album for presale, and each one has a different bonus song that's not on the others.
Swift fans are somewhat used to different versions of her albums. Her 2022 album, Midnights, came in six different colored vinyl editions, so fans had to pick their favorite or buy the same music over and over again just to add a slight variation to their collection.
See also: TikTok Is Changing the Way You Discover Music. Meet the Young Creators Making It Happen
And more famously, Swift remade her first six albums after music manager Scooter Braun sold the rights to that music to Disney family investment firm Shamrock Holdings in 2020 for an estimated $300 million to $450 million. Since then, she's encouraged fans to buy new versions of the same songs, dubbing them "Taylor's Version." So it's possible fans have already bought multiple copies of Swift's music.
But in this case, The Tortured Poets Department is a brand-new album release with multiple editions that vary by only one song. Fans who view themselves as collectors or completists would have to buy the same album three times to get the three different bonus tracks -- there's no way to buy just the bonus tracks separately.
Adding to the complexity, we live in a world dominated by music streaming services but also filled with multiple physical formats, including cassette tapes, compact discs and vinyl records.
Let's sing out the differences.
The Tortured Poets Department: What to know
The Tortured Poets Department is Taylor Swift's 11th studio album. The standard version of the album features 16 songs, including opener Fortnight, performed with Post Malone and now revealed as the album's first single, and a duet with Florence and the Machine called Florida!!! (Exclamation points are Swift's.)
The Tortured Poets Department is out in both digital and physical formats. Choosing that should probably be pretty simple -- how do you listen to music? Pick your format.
Four different editions
But then you'll have another choice to make. In addition to the standard 16-song version, there are three special editions, each containing its own bonus song. Two of the three are already unavailable, though you never know -- they could pop up for purchase again.
Here are all the choices:
The basic version: The cheapest way to get The Tortured Poets Department is to order the standard digital album. It costs $12 and includes the 16 basic tracks and no bonus tracks at all.
Editions with The Manuscript bonus track: If you do buy the basic version, you'll be missing out on a Swift bonus track called The Manuscript. Want that? You'll need to buy the album on cassette, CD or vinyl to get it. Those editions all remain available to preorder for delivery on April 19. The CD version is $13, cassette is $29 and the vinyl album is $35.
The Albatross version: There were also two versions of the album that contained the 16 tracks and bonus songs of their own but didn't have The Manuscript bonus track. One edition, featuring a bonus track called The Albatross, was available for presale until 11 a.m. PT on Feb. 26. So if you've missed that deadline, well, you might be out of luck.
The Bolter version: The Albatross and The Manuscript weren't the only bonus tracks. Entertainment Weekly reported that on Feb. 16, Swift announced yet another special edition of the album with a bonus song called The Bolter. Like The Albatross edition, it had a limited preorder time and doesn't appear to be available anymore.
There's no official word yet on whether the special editions of the album that feature The Albatross or The Bolter will be sold again. A representative for Swift didn't immediately respond to a request for comment.
We should note that no one has heard any of the songs yet, so we don't know what the titles The Manuscript, The Albatross or The Bolter refer to lyrically.
Fan reaction is mixed
Not all fans are thrilled with the prospect of buying different variants of the album over and over just for one new bonus song.
"This really is SOOO upsetting and I'm glad I'm not the only Swiftie who feels like we're getting taken advantage of," one fan wrote as a response to Swift's Facebook post about The Albatross edition.
Another fan agreed, writing, "I [understand] releasing one bonus album, but [multiple bonus tracks on multiple albums] is just a marketing ploy to get more money."
Some fans, of course, defended Swift. One pointed out that Swift's last few albums have offered different versions, writing, "Why is it such a big deal this time?"
Another wrote, "Well, this is her business. She's here to make money, not friends.
And others pointed out that there's no way the bonus tracks will stay secret once fans receive their preorders, even if the music ends up being shared illegally on YouTube.
Swift isn't the only artist to offer different bonus tracks on different versions of the same album. Olivia Rodrigo's 2023 album Guts, for example, came in four limited-edition versions on vinyl. Each album is a different color, and each color has its own bonus track. But because of Swift's enormous fan following, her bonus tracks have drawn more attention.
"I understand this more with Olivia," wrote one Reddit user. "Yes, she had a crazy successful debut. But she's still a new artist (and presumably was trying to avoid a sophomore slump). Taylor doing this is giving 'your-richest-friend-Venmo-requesting-you-$3.'"
There's no discount for buying multiple versions of Swift's new albums.
""",
    meta={"url":"https://www.cnet.com/tech/services-and-software/taylor-swifts-eras-tour-is-back-how-to-watch-the-original-concert-film/#google_vignette", "title":"Taylor Swift's New Album Is Here: Nabbing All the Bonus Tracks Will Cost You"}
),
Document(
    content="""
In the 18 months between 2022’s Midnights and Taylor Swift’s new album, The Tortured Poets Department, Swift managed to grow into an even bigger cultural force, releasing three deluxe reissues of Midnights, as well as Taylor’s Version re-recordings of 2014’s 1989 and 2010’s Speak Now. Between records, she embarked on the ongoing Eras Tour, a three-hour career retrospective that’s boosted local economies, grossed over a billion dollars, and attracted the level of media attention usually reserved for the deaths of well-liked presidents. Swift was already one of her generation’s biggest musicians when Midnights dropped, but everything that’s happened in the interim has cemented her as an astronomical superstar.

The Tortured Poets Department, Swift’s 11th original studio album, is billed as a concept album exploring the five stages of grief. She produced the album with regular collaborators Jack Antonoff, who produced Midnights, and the National’s Aaron Dessner, with whom she worked on her intimate pandemic albums, Folklore and Evermore. True to those credits, its muted aesthetic vacillates between Midnights’ downtempo pop and the delicate pop-folk of Folklore. It’s also a “secret double album” as Swift announced in the early hours of Friday morning; the complete 31-song set, subtitled The Anthology, clocks in at a whopping two hours and two minutes.

Given this is Swift’s first album since breaking up with boyfriend of six years Joe Alwyn, briefly dating the 1975’s Matty Healy, and entering into a headline-grabbing relationship with Kansas City Chiefs tight end Travis Kelce, there’s heartbreak and new love aplenty across Tortured Poets, as well as the kind of acidic self-loathing that’s noticeably crept into Swift’s songwriting since 2017’s Reputation. Here are eight big takeaways from The Tortured Poets Department.

Taylor Swift
Taylor Swift Releases New Album The Tortured Poets Department, Plus 15 More Songs: Listen and Read the Full Credits
The Anthology
As was the case with Midnights, The Tortured Poets Department was boosted with a deluxe reissue mere hours after its release. The Tortured Poets Department: The Anthology runs at over two hours long and includes “The Bolter,” “The Albatross,” “The Manuscript,” and “The Black Dog,” four songs that were previously available only on four variants of the album’s vinyl release. It’s the first extension of The Tortured Poets Department, but if Midnights is anything to go by, likely not the last: after that record’s 3am Edition we also got a Til Dawn edition and the Late Night Edition. Antonoff and Dessner produced all of the Anthology tracks, with Charli XCX collaborator Patrik Berger contributing to “I Look in People’s Windows.”

Still in the Folklorian Woods
With the release of 2020’s Folklore, Taylor Swift drew a line in the sand, making clear that the songs on that record were all rooted in fiction. That may not have been entirely true—“The Last Great American Dynasty” tells the story of Swift’s Rhode Island summer house, through to her purchase of it, and other songs seem to allude to her personal life—but it felt like a signal to fans not to read so deeply into everything she writes. (Swift describes this zone of her songwriting as “the folklorian woods.”) She seems to take a similar tack with Tortured Poets: opening track “Fortnight,” featuring Post Malone, spins a tale about exes who end up living next door to each other. Its placement as track one on the record suggests that we should accept Swift as a songwriter, not just a confessionalist.

Talkin’ ’Bout My Reputation
As on Reputation, Swift addresses her own public perception on Tortured Poets—understandable, given all the media attention she’s received in the past year. On the late album barnstormer “Who’s Afraid of Little Old Me?,” Swift sings about becoming callous in the face of speculation about her private life (“I was tame, I was gentle/Til the circus life made me mean”). On “I Can Do It With a Broken Heart,” she contrasts the cheery demeanor she adopts for fans on the Eras Tour with the abject sadness she feels post-breakup. In a moment that recalls the most memorable scene of Katy Perry’s Part of Me documentary—in which Perry finds out her husband has filed for divorce, bawls her eyes out, then resets to a cold smile as she rises to the stage—Swift cheerily exclaims “I’m miserable!/And nobody even knows!”


A Brief Inquiry Into Past Relationships
The Tortured Poets Department is sure to provide near-endless fodder for fans and rubberneckers hoping to glean insight into Swift’s personal life. Her lyrics are often assumed to be about real-life boyfriends, and many of the songs on this record seem to reference Healy (she refers to one partner as a “tattooed golden retriever” on the title track) and Alwyn, seemingly the subject of “So Long, London.” “The Alchemy,” which is heavy on football references, nods to her relationship with Kelce, with Swift singing about being “on a winning streak” with a new love. And it’s not just romantic relationships Swift addresses: The Anthology track “thanK you aIMee,” with its unsubtle title stylization, is widely rumored to be about longtime Swift nemesis Kim Kardashian.

Stuck Behind Bars
Allusions to prisons, asylums, and general ill mental health abound on Tortured Poets: On the album’s title track, she sings, “Everyone we know understands/Why it’s meant to be/’Cause we’re… crazy,” while on “But Daddy I Love Him,” she tells a disapproving father, “I know he’s crazy, but he’s the one I want.” “Fresh Out the Slammer” casts a maudlin relationship as a prison—Swift’s new crush is the first person she’s calling when she gets out—while she tells the titular character on “The Smallest Man Who Ever Lived”: “You deserve prison, but you won’t get time.” On “Who’s Afraid of Little Old Me?” the most vicious track here, she references gallows and likens her childhood to an “asylum,” and opening track “Fortnight” features the line, “I was supposed to be sent away/But they forgot to come and get me.” These references, a shade darker than the turns of phrase Swift is usually drawn to, seem to be in service of the album’s “tortured” vibe.

Taylor + the Machine
One of the highlights of Tortured Poets is “Florida!!!,” a collaboration with stalwart British pop musician Florence Welch of Florence and the Machine. Swift may have been dinged for relegating Lana Del Rey to backing vocalist status on Midnights’ “Snow on the Beach” (which Del Rey later said was her own choice), but there’s no way the same will happen with “Florida!!!”—Welch gets two full verses here, and later trades lines with Swift. Featuring a doomy aesthetic and huge, window-rattling drums, it feels more like a Welch track that Swift accidentally wandered into. Which is no knock: It’s the loudest, most theatrical moment on an album that otherwise trades punchy showstoppers for quieter, more meditative moods.
""",
    meta={"url":"https://pitchfork.com/thepitch/takeaways-from-taylor-swift-new-album-the-tortured-poets-department/", "title":"8 Takeaways From Taylor Swift’s New Album The Tortured Poets Department"})
]

## Index Documents to your DocumentStore

In [ ]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack_integrations.components.embedders.amazon_bedrock import AmazonBedrockDocumentEmbedder

document_store = InMemoryDocumentStore()

indexing_pipeline = Pipeline()
indexing_pipeline.add_component("cleaner", DocumentCleaner())
indexing_pipeline.add_component("splitter", DocumentSplitter(split_by="word", split_length=512, split_overlap=50))
indexing_pipeline.add_component("embedder", AmazonBedrockDocumentEmbedder(model="amazon.titan-embed-text-v1"))
indexing_pipeline.add_component("writer", DocumentWriter(document_store=document_store))
indexing_pipeline.connect("cleaner", "splitter")
indexing_pipeline.connect("splitter", "embedder")
indexing_pipeline.connect("embedder", "writer")

indexing_pipeline.run({"cleaner": {"documents": documents}})

Creating embeddings: 100%|██████████| 14/14 [00:06<00:00,  2.18it/s]


{'writer': {'documents_written': 14}}

## Creating the Initial Pipeline Components

First, define a prompt instructing the LLM to respond with the text `"no_answer"` if the provided documents do not offer enough context to answer the query. Next, initialize a [PromptBuilder](https://docs.haystack.deepset.ai/v2.0/docs/promptbuilder) with that prompt. It's crucial that the LLM replies with `"no_answer"` as you will use this keyword to indicate that the query should be directed to the alternative web search route.

As the LLM, you will use an [AmazonBedrockGenerator](https://docs.haystack.deepset.ai/docs/amazonbedrockgenerator) with the `amazon.titan-text-express-v1` model.

> The provided prompt works effectively with the `amazon.titan-text-express-v1` model. If you prefer to use models, you may need to update the prompt to provide clearer instructions to your model.

Find all Amazon Bedrock model IDs [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns)

In [ ]:
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack_integrations.components.generators.amazon_bedrock import AmazonBedrockGenerator
from haystack_integrations.components.embedders.amazon_bedrock import AmazonBedrockTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

prompt_template = """
Answer the following query given the douments.
If the answer is not contained within the documents reply with 'no_answer'

Documents:
{% for document in documents %}
  {{document.content}}
{% endfor %}

Query: {{query}}
"""

bedrock_model = 'amazon.titan-text-express-v1' # @param ["amazon.titan-text-express-v1", "amazon.titan-text-lite-v1", "meta.llama3-8b-instruct-v1:0", "meta.llama3-70b-instruct-v1:0", "mistral.mixtral-8x7b-instruct-v0:1", "anthropic.claude-instant-v1", "anthropic.claude-v1", "anthropic.claude-v2","anthropic.claude-v2:1", "meta.llama2-13b-chat-v1", "meta.llama2-70b-chat-v1", "ai21.j2-mid-v1", "ai21.j2-ultra-v1"]

rag_pipeline = Pipeline()
rag_pipeline.add_component("text_embedder", AmazonBedrockTextEmbedder(model="amazon.titan-embed-text-v1"))
rag_pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store, top_k=5))
rag_pipeline.add_component("prompt_builder", PromptBuilder(template=prompt_template))
rag_pipeline.add_component("generator", AmazonBedrockGenerator(model=bedrock_model, max_length=500))
rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "generator")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

🚅 Components
  - text_embedder: AmazonBedrockTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - generator: AmazonBedrockGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)

In [ ]:
query="What's Tortured Poets Department album about?"
# query="When is Taylor Swift's next concert?" # [no_answer]
# query="When did Taylor Swift release her Red album?" # [no_answer]

rag_pipeline.run({"text_embedder":{"text": query}, "prompt_builder":{"query": query}})

{'generator': {'replies': ["Fans have theorized that the upcoming album will detail Swift's split from actor Joe Alwyn, whom she dated for six years before splitting in April 2023. Not only do the song titles include references to the English actor and their former relationship, but Swift recently teased that the album is about heartbreak as she shared five exclusive playlists with Apple Music.\nAccording to a press release, with each playlist, Swift has chosen songs from her own catalog that fit each stage of a breakup: denial, anger, bargaining, depression and acceptance."]}}

## Initializing the Web Search Components

Initialize the necessary components for a web-based RAG application. Along with a `PromptBuilder` and an `OpenAIGenerator`, you will need a [SerperDevWebSearch](https://docs.haystack.deepset.ai/v2.0/docs/serperdevwebsearch) to retrieve relevant documents for the query from the web.

> If desired, you can define a different prompt or utilize a different [Generator](https://docs.haystack.deepset.ai/v2.0/docs/generators) for the web-based RAG branch of the pipeline.

In [ ]:
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.components.websearch.serper_dev import SerperDevWebSearch

web_search_prompt_template="""
Answer the following query given the documents retrieved from the web.
Your answer shoud indicate that your answer was generated from websearch.

Documents:
{% for document in documents %}
  {{document.content}}
{% endfor %}

Query: {{query}}
"""

websearch = SerperDevWebSearch()
prompt_builder_for_websearch = PromptBuilder(template=web_search_prompt_template)
llm_for_websearch = AmazonBedrockGenerator(model=bedrock_model, max_length=500)

## Creating the ConditionalRouter

[ConditionalRouter](https://docs.haystack.deepset.ai/v2.0/docs/conditionalrouter) is the component that handles data routing on specific conditions. You need to define a `condition`, an `output`, an `output_name` and an `output_type` for each route. Each route that the `ConditionalRouter` creates acts as the output of this component and can be connected to other components in the same pipeline.  

In this case, you need to define two routes:
- If the LLM replies with the `"no_answer"` keyword, the pipeline should perform web search. It means that you will put the original `query` in the output value to pass to the next component (in this case the next component will be the `SerperDevWebSearch`) and the output name will be `go_to_websearch`.
- Otherwise, the given documents are enough for an answer and pipeline execution ends here. Return the LLM reply in the output named `answer`.

In [ ]:
from haystack.components.routers import ConditionalRouter

routes = [
    {
        "condition": "{{'no_answer' in replies[0]}}",
        "output": "{{query}}",
        "output_name": "go_to_websearch",
        "output_type": str,
    },
    {
        "condition": "{{'no_answer' not in replies[0]}}",
        "output": "{{replies[0]}}",
        "output_name": "answer",
        "output_type": str,
    },
]

router = ConditionalRouter(routes)

## Building the Pipeline

Add all components to your pipeline and connect them. `go_to_websearch` output of the `router` should be connected to the `websearch` to retrieve documents from the web and also to `prompt_builder_for_websearch` to use in the prompt.

In [ ]:
from haystack import Pipeline

pipe_with_fallback = Pipeline()

pipe_with_fallback.add_component("text_embedder", AmazonBedrockTextEmbedder(model="amazon.titan-embed-text-v1"))
pipe_with_fallback.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store, top_k=5))
pipe_with_fallback.add_component("prompt_builder", PromptBuilder(template=prompt_template))
pipe_with_fallback.add_component("generator", AmazonBedrockGenerator(model=bedrock_model, max_length=500))

pipe_with_fallback.add_component("router", router)
pipe_with_fallback.add_component("websearch", websearch)
pipe_with_fallback.add_component("prompt_builder_for_websearch", prompt_builder_for_websearch)
pipe_with_fallback.add_component("generator_for_websearch", llm_for_websearch)


pipe_with_fallback.connect("text_embedder.embedding", "retriever.query_embedding")
pipe_with_fallback.connect("retriever", "prompt_builder.documents")
pipe_with_fallback.connect("prompt_builder", "generator")
pipe_with_fallback.connect("generator.replies", "router.replies")
pipe_with_fallback.connect("router.go_to_websearch", "websearch.query")
pipe_with_fallback.connect("router.go_to_websearch", "prompt_builder_for_websearch.query")
pipe_with_fallback.connect("websearch.documents", "prompt_builder_for_websearch.documents")
pipe_with_fallback.connect("prompt_builder_for_websearch", "generator_for_websearch")

🚅 Components
  - text_embedder: AmazonBedrockTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - generator: AmazonBedrockGenerator
  - router: ConditionalRouter
  - websearch: SerperDevWebSearch
  - prompt_builder_for_websearch: PromptBuilder
  - generator_for_websearch: AmazonBedrockGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)
  - generator.replies -> router.replies (List[str])
  - router.go_to_websearch -> websearch.query (str)
  - router.go_to_websearch -> prompt_builder_for_websearch.query (str)
  - websearch.documents -> prompt_builder_for_websearch.documents (List[Document])
  - prompt_builder_for_websearch.prompt -> generator_for_websearch.prompt (str)

### Visualize the Pipeline

To understand how you formed this pipeline with conditional routing, use [draw()](https://docs.haystack.deepset.ai/v2.0/docs/drawing-pipeline-graphs) method of the pipeline. If you're running this notebook on Google Colab, the generated file will be saved in "Files" section on the sidebar or you can call `Image.open()`:

In [ ]:
from PIL import Image

pipe_with_fallback.draw("pipe.png")
Image.open("pipe.png")

## Running the Pipeline!

In the `run()`, pass the query to the `prompt_builder` and the `router`. In real life applications, `documents` will be provided by a [Retriever](https://docs.haystack.deepset.ai/v2.0/docs/retrievers) but to keep this example simple, you will provide the defined `documents` to the `prompt_builder`.

In [ ]:
query="What's Tortured Poets Department album about?"
query="When is Taylor Swift's next concert?" # [no_answer]
# query="When did Taylor Swift release her Red album?" # [no_answer])

result = pipe_with_fallback.run({"text_embedder":{"text": query}, "prompt_builder": {"query": query}, "router": {"query": query}})
result

{'websearch': {'links': ['https://www.taylorswift.com/tour/',
   'https://www.songkick.com/artists/217815-taylor-swift/calendar',
   'https://www.ticketmaster.com/taylor-swift-tickets/artist/1094215',
   'https://www.oregonlive.com/entertainment/2024/03/when-and-where-is-taylor-swifts-next-show-how-to-get-the-cheapest-tickets-for-her-viral-eras-tour.html',
   'https://www.songkick.com/artists/217815-taylor-swift',
   'https://www.sportingnews.com/us/nfl/news/taylor-swift-tour-2024-tickets-prices-us-schedule-dates/612e97acae959d6de55ea896',
   'https://www.usatoday.com/story/entertainment/music/2023/08/03/taylor-swift-adds-north-american-concert-dates-to-eras-tour/70520449007/',
   'https://www.oregonlive.com/events/2024/05/when-is-taylor-swifts-next-us-tour-date-how-to-get-the-cheapest-tickets-for-eras-tour-see-her-met-gala-looks.html',
   'https://www.hindustantimes.com/entertainment/music/taylor-swift-eras-tour-full-list-of-dates-and-venues-of-upcoming-shows-101695626500588.html']},


## What's next

🎉 Congratulations! You've built a pipeline with conditional routing! You can now customize the condition for your specific use case and create a custom Haystack 2.0 pipeline to meet your needs.

If you liked this tutorial, there's more to learn about Haystack 2.0:
- [Creating Your First QA Pipeline with Retrieval-Augmentation](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline)
- [Model-Based Evaluation of RAG Pipelines](https://haystack.deepset.ai/tutorials/35_model_based_evaluation_of_rag_pipelines)

To stay up to date on the latest Haystack developments, you can [sign up for our newsletter](https://landing.deepset.ai/haystack-community-updates?utm_campaign=developer-relations&utm_source=tutorial&utm_medium=conditional-router) or [join Haystack discord community](https://discord.gg/haystack).

Thanks for reading!